In [1]:
import pandas as pd
import networkx as nx
import igraph as ig
import numpy as np
import matplotlib.pyplot as plt

import pickle

from community import community_louvain as louvain # pip install python-louvain


CUSTOM_FORMAT = {"font_size": 8,
                "node_size": 80, "node_color": "#ff0000", "edgecolors": "#911a07",
                "width": 0.4, "edge_color": "black"}

In [2]:
with open("../../state_files/PyPi Dataframe V4.pickle", "rb") as f:
    df = pickle.load(f)

G = nx.read_gexf('../../state_files/PyPi Network V4.gexf')
G_ig = ig.Graph.TupleList(G.edges(), directed=True) # Mejor manera que la que teníamos. Conserva nombres.

# Comunidades

In [3]:
def clusters_to_list(clusters, G) -> dict:
    """
    Arma un diccionario con los nombres de los nodos
    como keys y el id del cluster como valor.
    """
    dic = {}
    clusters_list = []
    for i, cluster in enumerate(sorted(list(clusters), key = len, reverse = True)):
        nodes_in_cluster = []
        for node in cluster:
            nodes_in_cluster.append(G.vs[node]['_nx_name'])
        clusters_list.append(nodes_in_cluster)
    return clusters_list

In [3]:
# Louvain no se puede hacer porque no funciona para redes dirigidas.
# Fast Greedy tampoco se puede hacer en redes dirigidas.

infomap = True
edge_betweenes = False

# Infomap.
if infomap:
    try:
        with open("com_im.pickle", "rb") as f:
            com_im = pickle.load(f)

    except FileNotFoundError:
        com_im = G_ig.community_infomap()
        with open("com_im.pickle", "wb") as f:
            pickle.dump(com_im, f)
        print("Infomap done.")

# Edge Betweeness
if edge_betweenes:
    try:
        with open("com_bt.pickle", "rb") as f:
            com_bt = pickle.load(f)
    except FileNotFoundError:
        com_bt = G_ig.community_edge_betweenness(clusters = None, directed = True, weights = None)
        with open("com_bt.pickle", "wb") as f:
            pickle.dump(com_bt, f)
        print("Edge Betweeness done.")

### Código de Eitu

In [ ]:
clusters = com_fg.as_clustering()
communities = clusters_to_list(clusters, G_ig)
for i in range(15):
    print(len(communities[i]))
nodes_to_plot = communities[10]
G_com = G.subgraph(nodes_to_plot)
nx.draw(G_com, labels={i: str(i) for i in range(len(list(G_com.nodes())))}, **CUSTOM_FORMAT)
fig, axs = plt.subplots(2, 2)
axs = axs.flatten()
for i, com_ix in enumerate(range(8, 12)):
    nodes_to_plot = communities[com_ix]
    G_com = G.subgraph(nodes_to_plot)
    nx.draw_kamada_kawai(G_com, ax=axs[i], **CUSTOM_FORMAT)